In [2]:
import sqlite3
import datetime
import pandas as pd
conn = sqlite3.connect("stock.sqlite")
cur = conn.cursor()
query = "select distinct Ticker from stock_table_hr"
r_df = pd.read_sql(query,conn)
list_db = r_df['Ticker'].values.tolist()
print(list_db)
print(len(list_db))

['AAV.BK', 'ACE.BK', 'ADVANC.BK', 'AMATA.BK', 'AOT.BK', 'AP.BK', 'AWC.BK', 'BAM.BK', 'BANPU.BK', 'BBL.BK', 'BCH.BK', 'BCP.BK', 'BCPG.BK', 'BDMS.BK', 'BEC.BK', 'BEM.BK', 'BGRIM.BK', 'BH.BK', 'BLA.BK', 'BTS.BK', 'BYD.BK', 'CBG.BK', 'CENTEL.BK', 'CHG.BK', 'CK.BK', 'CKP.BK', 'COM7.BK', 'CPALL.BK', 'CPF.BK', 'CPN.BK', 'CRC.BK', 'DELTA.BK', 'DOHOME.BK', 'DTAC.BK', 'EA.BK', 'EGCO.BK', 'EPG.BK', 'ESSO.BK', 'FORTH.BK', 'GLOBAL.BK', 'GPSC.BK', 'GULF.BK', 'GUNKUL.BK', 'HANA.BK', 'HMPRO.BK', 'INTUCH.BK', 'IRPC.BK', 'IVL.BK', 'JAS.BK', 'JMART.BK', 'JMT.BK', 'KBANK.BK', 'KCE.BK', 'KEX.BK', 'KKP.BK', 'KTB.BK', 'KTC.BK', 'LH.BK', 'MEGA.BK', 'MINT.BK', 'MTC.BK', 'NEX.BK', 'ONEE.BK', 'OR.BK', 'ORI.BK', 'OSP.BK', 'PLANB.BK', 'PSL.BK', 'PTG.BK', 'PTT.BK', 'PTTEP.BK', 'PTTGC.BK', 'QH.BK', 'RATCH.BK', 'RBF.BK', 'RCL.BK', 'SABUY.BK', 'SAWAD.BK', 'SCB.BK', 'SCC.BK', 'SCGP.BK', 'SINGER.BK', 'SPALI.BK', 'SPRC.BK', 'STA.BK', 'STARK.BK', 'STGT.BK', 'TCAP.BK', 'THANI.BK', 'THG.BK', 'TIDLOR.BK', 'TIPH.BK', 'TISCO.B

In [1]:
# Run this app with `python app.py` and
# visit http://127.0.0.1:8050/ in your web browser.

from jupyter_dash import JupyterDash
from dash import Dash, html, dcc, Input, Output, callback , State, ctx, dash_table
import plotly.express as px
import pandas as pd

import numpy as np
import pandas as pd
import plotly
from plotly.subplots import make_subplots

#Data Source
import yfinance as yf

#Data viz
import plotly.graph_objs as go
import sqlite3
import datetime
from datetime import date
import dateutil.relativedelta

app = JupyterDash(__name__)

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options

def table(ticker):
    con = sqlite3.connect("stock.sqlite")
    cur = con.cursor()
    df = pd.read_sql("SELECT * FROM stock_financial where Ticker = '"+ticker+"';",con)
    column = df['Year'].values.tolist()
    df = df.transpose()
    df.columns = column
    df = df.drop("Ticker")
    df = df.drop("Year")
    head = df.index.values.tolist()
    df.insert(0, "Financial Information", head, True)
    return df

def plot(ticker, interval):
    now = datetime.datetime.now()
    last_month = now + dateutil.relativedelta.relativedelta(months=-1)
    seven_month = now + dateutil.relativedelta.relativedelta(months=-7)
    ten_year = now + dateutil.relativedelta.relativedelta(years=-10)
    conn = sqlite3.connect("stock.sqlite")
    cur = conn.cursor()
    query1 = "select * from stock_table_hr where `ticker` == '%s' and datetime > '%s' and datetime < '%s'" % (ticker, last_month, now)
    query2 = "select * from stock_table_d where `ticker` == '%s' and datetime > '%s' and datetime < '%s'" % (ticker, seven_month, now)
    query3 = "select * from stock_table_mo where `ticker` == '%s' and datetime > '%s' and datetime < '%s'" % (ticker, ten_year, now)
    Hr = pd.read_sql(query1,conn)
    Day = pd.read_sql(query2,conn)
    Mo = pd.read_sql(query3,conn)
    #declare figure
    fig = go.Figure()
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, 
                vertical_spacing=0.1, subplot_titles=('OHLC', 'Volume','MACD'), 
                row_width=[0.2, 0.7, 0.7])

    #Candlestick

    # fig.update_xaxes( rangebreaks=[ dict(values = timegap_hr , dvalue = 3600000)])
    if interval == "Hour":
        Hr['time'] = pd.to_datetime(Hr['Datetime'])
        Hr['MA50'] = Hr['Close'].rolling(window = 50 , min_periods = 0).mean()
        Hr['MA200'] = Hr['Close'].rolling(window = 200 , min_periods = 0).mean()
        Hr['EMA12'] = Hr['Close'].ewm(span=12, adjust=False, min_periods = 0).mean()
        Hr['EMA26'] = Hr['Close'].ewm(span=26, adjust=False, min_periods = 0).mean()
        Hr.set_index(Hr['time'],inplace = True)
        macd = Hr['EMA12'] - Hr['EMA26']
        signal = macd.ewm(span=9, adjust=False).mean()
        fig.add_trace(go.Candlestick(x=Hr.Datetime,
                        open=Hr.Open,
                        high=Hr.High,
                        low=Hr.Low,
                        close=Hr.Close, 
                        name = ticker,),row=1, col=1)
        fig.add_trace(go.Scatter(name = 'MA50',x=Hr.Datetime,y=Hr['MA50']),row=1, col=1)
        fig.add_trace(go.Scatter(name = 'MA200',x=Hr.Datetime,y=Hr['MA200']),row=1, col=1)
        fig.add_trace(go.Scatter(name = 'EMA12',x=Hr.Datetime,y=Hr['EMA12']),row=1, col=1)
        fig.add_trace(go.Scatter(name = 'EMA26',x=Hr.Datetime,y=Hr['EMA26']),row=1, col=1)
        fig.add_trace(go.Bar(x=Hr.Datetime, y=Hr.Volume,showlegend=False), row=2, col=1)
        fig.add_trace(go.Bar(name = 'MACD',x=Hr.Datetime,y=signal),row=3, col=1)

        df_resample_hr = Hr.resample('H').max()
        merged_index_hr  = Hr.index.append(df_resample_hr.index)
        timegap_hr = merged_index_hr[~merged_index_hr.duplicated(keep = False)]

        fig.update_xaxes(rangebreaks=[dict(values=timegap_hr, dvalue=3600000)])
        
    elif interval == "Day":
        Day['time'] = pd.to_datetime(Day['Datetime'])
        Day['MA50'] = Day['Close'].rolling(window = 50 , min_periods = 0).mean()
        Day['MA200'] = Day['Close'].rolling(window = 200 , min_periods = 0).mean()
        Day['EMA12'] = Day['Close'].ewm(span=12, adjust=False, min_periods = 0).mean()
        Day['EMA26'] = Day['Close'].ewm(span=26, adjust=False, min_periods = 0).mean()
        macd = Day['EMA12'] - Day['EMA26']
        signal = macd.ewm(span=9, adjust=False).mean()
        Day.set_index(Day['time'],inplace = True)
        fig.add_trace(go.Candlestick(x=Day.Datetime,
                        open=Day.Open,
                        high=Day.High,
                        low=Day.Low,
                        close=Day.Close, 
                        name = ticker,),row=1, col=1)
        fig.add_trace(go.Scatter(name = 'MA50',x=Day.Datetime,y=Day['MA50']),row=1, col=1)
        fig.add_trace(go.Scatter(name = 'MA200',x=Day.Datetime,y=Day['MA200']),row=1, col=1)
        fig.add_trace(go.Scatter(name = 'EMA12',x=Day.Datetime,y=Day['EMA12']),row=1, col=1)
        fig.add_trace(go.Scatter(name = 'EMA26',x=Day.Datetime,y=Day['EMA26']),row=1, col=1)
        fig.add_trace(go.Bar(x=Day.Datetime, y=Day.Volume,showlegend=False), row=2, col=1)
        fig.add_trace(go.Bar(name = 'MACD',x=Day.Datetime,y=signal),row=3, col=1)

    
        df_resample_day = Day.resample('D').max()
        merged_index_day  = Day.index.append(df_resample_day.index)
        timegap_day = merged_index_day[~merged_index_day.duplicated(keep = False)]
        fig.update_xaxes(rangebreaks=[dict(values=timegap_day)])

    elif interval == "Month":
        Mo['time'] = pd.to_datetime(Mo['Datetime'])
        Mo['MA50'] = Mo['Close'].rolling(window = 50 , min_periods = 0).mean()
        Mo['MA200'] = Mo['Close'].rolling(window = 200 , min_periods = 0).mean()
        Mo['EMA12'] = Mo['Close'].ewm(span=12, adjust=False, min_periods = 0).mean()
        Mo['EMA26'] = Mo['Close'].ewm(span=26, adjust=False, min_periods = 0).mean()
        Mo.set_index(Mo['time'],inplace = True)
        macd = Mo['EMA12'] - Mo['EMA26']
        signal = macd.ewm(span=9, adjust=False).mean()
        fig.add_trace(go.Candlestick(x=Mo.Datetime,
                        open=Mo.Open,
                        high=Mo.High,
                        low=Mo.Low,
                        close=Mo.Close, 
                        name = ticker,),row=1, col=1)
        fig.add_trace(go.Scatter(name = 'MA50',x=Mo.Datetime,y=Mo['MA50']),row=1, col=1)
        fig.add_trace(go.Scatter(name = 'MA200',x=Mo.Datetime,y=Mo['MA200']),row=1, col=1)
        fig.add_trace(go.Scatter(name = 'EMA12',x=Mo.Datetime,y=Mo['EMA12']),row=1, col=1)
        fig.add_trace(go.Scatter(name = 'EMA26',x=Mo.Datetime,y=Mo['EMA26']),row=1, col=1)
        fig.add_trace(go.Bar(x=Mo.Datetime, y=Mo.Volume,showlegend=False), row=2, col=1)
        fig.add_trace(go.Bar(name = 'MACD',x=Mo.Datetime,y=signal),row=3, col=1)

    fig.update_xaxes(rangemode="nonnegative")
    fig.update_yaxes(fixedrange=False)
    fig.update_layout(hovermode = "x")
    fig.update(layout_xaxis_rangeslider_visible=False)
    return fig

def getLastDate(period,ticker):
    conn = sqlite3.connect("stock.sqlite")
    cur = conn.cursor()
    # Query last element of stock in database
    if period == 'Hour':
        query = "SELECT * FROM stock_table_hr WHERE `ticker` = '%s'" % ticker
    elif period == 'Day':
        query = "SELECT * FROM stock_table_d WHERE `ticker` = '%s'" % ticker
    elif period == 'Mount':
        query = "SELECT * FROM stock_table_mo WHERE `ticker` = '%s'" % ticker
    else:
        return False
    r_df = pd.read_sql(query, conn)
    # Cut data to get only datatime
    last = r_df.tail(1).Datetime.to_string().split()
    LastDate = last[1].split()[0].split('-')
    cur.close()
    return LastDate,r_df

def getDiffDay(period,ticker):
    LastDate,r_df = getLastDate(period,ticker)
    if LastDate == False:
        return False
    # Get datetime for now
    x = datetime.datetime.now()
    count = 0
    DayM = 0
    DayMo365 = {'1':31,'2':28,'3':31,'4':30,'5':31,'6':30,'7':31,'8':31,'9':30,'10':31,'11':30,'12':31}
    DiffMo = int(x.month) - int(LastDate[1])
    DiffYe = int(x.year) - int(LastDate[0])
    # Get differend day for dowload stock
    if DiffYe == 0:
        if DiffMo == 0:
            DiffDay = int(x.day) - int(LastDate[2])
            if DiffDay != 0:pass
        elif DiffMo != 0 :
            for u in range(DiffMo):
                DayM = DayM + DayMo365[str(int(LastDate[1])+count)]
                count += 1
            DiffDay = DayM - int(LastDate[2]) + int(x.day)
    elif DiffYe != 0:
        dayly = 0
        dayn = 0
        for j in range(1,int(LastDate[1])):
            dayly = dayly + DayMo365[str(j)]
        for i in range(1,int(x.month)):
            dayn = dayn + DayMo365[str(i)]
        DiffDay = (365*DiffYe) - dayly + dayn - int(LastDate[2]) + int(x.day)   
    DiffDay = str(DiffDay) + 'd'
    return DiffDay
    
    
def update(period,ticker):
    LastDate,r_df = getLastDate(period,ticker)
    DiffDay = getDiffDay(period,ticker)
    if DiffDay == False:
        return False
    down = 0
    count = 0
    conn = sqlite3.connect("stock.sqlite")
    # Select period to download
    if period == 'Hour':data = yf.download(tickers=ticker, period=DiffDay, interval='1h')
    elif period == 'Day':data = yf.download(tickers=ticker, period=DiffDay, interval='1d')
    elif period == 'Mount':data = yf.download(tickers=ticker, period=DiffDay, interval='1mo')
    # Get number of extra stock
    for i in data.index.day:
        if data.index.year[count] == int(LastDate[0]):
            if data.index.month[count] == int(LastDate[1]):
                if i == int(LastDate[2])+1 or i == int(LastDate[2])+2 or i == int(LastDate[2])+3:
                    break
        count += 1
    ok = r_df.tail(1).Datetime.to_string().split()[2]
    if ok == '10:00:00':down = 5
    elif ok == '11:00:00':down = 4
    elif ok == '12:00:00':down = 3
    elif ok == '14:00:00':down = 2
    elif ok == '15:00:00':down = 1
    elif ok == '16:00:00':down = 0
    # Cut extra stock off
    count = count - down
    data['ticker'] = ticker
    data = data.iloc[count:,:]
    data.index.rename('Datetime', inplace=True)
    # Save to sqlite
    # Select period to download
    if period == 'Hour':data.to_sql('stock_table_hr',con=conn,if_exists='append',index=True)
    elif period == 'Day':data.to_sql('stock_table_d',con=conn,if_exists='append',index=True)
    elif period == 'Mount':data.to_sql('stock_table_mo',con=conn,if_exists='append',index=True)
    return data
        
def stock_name():
    conn = sqlite3.connect("stock.sqlite")
    cur = conn.cursor()
    query = "select distinct Ticker from stock_table_hr"
    r_df = pd.read_sql(query,conn)
    list_db = r_df['Ticker'].values.tolist()
    return list_db

def download_ticker(period,ticker):
        conn = sqlite3.connect("stock.sqlite")
        cur = conn.cursor()
        try:
            # Select period to download
            if period == 'Hour':
                data = yf.download(tickers=ticker, period='2y', interval='1h',progress=False)
            elif period == 'Day':
                data = yf.download(tickers=ticker, period='max', interval='1d',progress=False)
            elif period == 'Mount':
                data = yf.download(tickers=ticker, period='max', interval='1mo',progress=False)
            else:
                data = []
            # Save to sqlite3
            # data.to_sql('stock_table',con=conn,if_exists='append',index=True)
            # return data to ploting graph
            if data.empty:
                raise ValueError
            return data
        except:
            return []

######DASH######
app.layout = html.Div(children=[
    html.Div([
        html.H1(children='Stock Radar',
                style={
                    'textAlign': 'center',
                    }
        ),
        html.Div([
            "Stock : ",
            dcc.Input(id='ticker-input', value='', type='text'),
            html.Button(id='submit-button', n_clicks=0, children='Submit'),
        ],
            style={
                'margin-bottom': 20,
            }
        ),
        dcc.Dropdown(['Hour', 'Day', 'Month'],
            searchable=False,
            id='stock-dropdown',
            style={
                'margin-bottom': 20,
            }
        ),
        html.Div(id='output-div'),
        html.Div(id="disp-ticker"),
        html.Button('Show Table', id='show-table-button'),
        ],
        style={
            'margin-top' : 50,
            'margin-right': 100,
            'margin-left': 100
        }
    ),
    html.Div([
        dcc.Graph(
            id='graph',
            figure=plot('','Hour')
        )],
        style={
                'margin-right': 20,
                'margin-left': 20
        }
    ),

    dcc.ConfirmDialog(
        id='popup',
        message='Not found that tickers',
        displayed=False,
    ),
    dcc.ConfirmDialog(
        id='popup_dl',
        message='Downloaded',
        displayed=False,
    ),
    dcc.ConfirmDialog(
        id='popup_fa',
        message='False to download for this ticker',
        displayed=False,
    ),
    html.Div(id='table-container'),


])

@app.callback([Output('graph','style'),Output('popup', 'displayed')],
              [Input('submit-button', 'n_clicks')],
              [State('ticker-input', 'value')])

def update_popup_and_graph(n_clicks, value):
    list_db = stock_name()
    value = value.upper()
    if n_clicks:
        if value.strip() == '' or (value.strip() not in list_db):
            return plot('','Hour'),True
        else:
            return plot(value, 'Hour'),False
    return plot('', 'Hour'),False

#return graph from ticker input
@app.callback(Output('graph','figure'),
              [Input('stock-dropdown', 'value'), 
              Input('submit-button', 'n_clicks')],
              [State('ticker-input', 'value')],)
def update_graph(stock_dropdown, submit_clicks, ticker_input):
    global index
    if stock_dropdown != None:
        return dropdown_output(stock_dropdown, ticker_input)
    elif submit_clicks is not None:
        index = ticker_input
        return update_output(ticker_input)   

def update_output(ticker_input):
    list_db = stock_name()
    value = ticker_input.upper()
    if value.strip() == '' or (value.strip() not in list_db):
            fig = plot('','Hour')
            return fig
    else:
        for i in ['Hour','Day','Mount']:
            update(i,value)
        value = value.upper()
        fig = plot(value,'')
        return fig

def dropdown_output(stock_dropdown ,ticker_input):
    value = str(ticker_input).upper()
    fig = plot(value, stock_dropdown)
    return fig


#
@app.callback(
    [Output("popup_dl", "displayed"),Output("popup_fa", "displayed")],
    [Input("popup", "submit_n_clicks")],
    [State('ticker-input', 'value')]
)
def execute_function(submit_n_clicks ,value):
    data = download_ticker('Hour',value)
    if submit_n_clicks and data != []:
        return True,False
    elif submit_n_clicks and data == []:
        return False,True
    return False,False

# reset value dropdown
@app.callback(Output('stock-dropdown', 'value'),
              Output("disp-ticker", "children"),
              [Input('submit-button', 'n_clicks')],
              [State('ticker-input', 'value')]
             )
def reset_dropdown(n_clicks, value):
    return None, value.upper()

#show table when press button
@app.callback(
    Output('table-container', 'children'),
    [Input('show-table-button', 'n_clicks'),
    Input('submit-button', 'n_clicks')]
)
def show_table(n_clicks1, n_clicks2):
    print(index)
    df = table(index)
    if "submit-button" == ctx.triggered_id:
        return []

    elif "show-table-button" == ctx.triggered_id: 
        return dash_table.DataTable(df.to_dict('records'), [{"name": i, "id": i} for i in df.columns],
            id='table-financial',
            style_cell={
                'whiteSpace': 'normal',
                'height': 'auto'},
                fill_width=False
        )




app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


PyQT

In [2]:
import sys

from PyQt5.QtWebEngineWidgets import QWebEnginePage
from PyQt5.QtWidgets import QApplication
from PyQt5.QtCore import QUrl
from PyQt5.QtWebEngineWidgets import QWebEngineView

url = 'http://127.0.0.1:8050/'

app = QApplication(sys.argv)

# QWebEngineView
browser = QWebEngineView()
browser.load(QUrl(url))
browser.show()

sys.exit(app.exec_())









PTT.BK
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
PTT.BK
PTT.BK
PTT.BK
PTT.BK
PTT.BK


In [2]:
import sys

from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtCore import QUrl
from PyQt5.QtWebEngineWidgets import QWebEnginePage, QWebEngineView
from PyQt5.QtWidgets import QApplication

url = 'http://127.0.0.1:8050/'


class Ui_MainWindow(object):
    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        MainWindow.resize(1130, 781)
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        self.verticalLayoutWidget = QtWidgets.QWidget(self.centralwidget)
        self.verticalLayoutWidget.setGeometry(QtCore.QRect(0, 0, 241, 781))
        self.verticalLayoutWidget.setObjectName("verticalLayoutWidget")
        self.verticalLayout = QtWidgets.QVBoxLayout(self.verticalLayoutWidget)
        self.verticalLayout.setContentsMargins(0, 0, 0, 0)
        self.verticalLayout.setObjectName("verticalLayout")
        self.label = QtWidgets.QLabel(self.verticalLayoutWidget)
        self.label.setStyleSheet("font: 75 10pt \"MS Shell Dlg 2\";")
        self.label.setAlignment(QtCore.Qt.AlignCenter)
        self.label.setObjectName("label")
        self.verticalLayout.addWidget(self.label)
        self.pushButton_4 = QtWidgets.QPushButton(self.verticalLayoutWidget)
        self.pushButton_4.setEnabled(True)
        sizePolicy = QtWidgets.QSizePolicy(QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Fixed)
        sizePolicy.setHorizontalStretch(0)
        sizePolicy.setVerticalStretch(0)
        sizePolicy.setHeightForWidth(self.pushButton_4.sizePolicy().hasHeightForWidth())
        self.pushButton_4.setSizePolicy(sizePolicy)
        self.pushButton_4.setIconSize(QtCore.QSize(20, 20))
        self.pushButton_4.setObjectName("pushButton_4")
        self.verticalLayout.addWidget(self.pushButton_4)
        self.pushButton = QtWidgets.QPushButton(self.verticalLayoutWidget)
        self.pushButton.setObjectName("pushButton")
        self.verticalLayout.addWidget(self.pushButton)
        self.pushButton_2 = QtWidgets.QPushButton(self.verticalLayoutWidget)
        self.pushButton_2.setObjectName("pushButton_2")
        self.verticalLayout.addWidget(self.pushButton_2)
        self.pushButton_3 = QtWidgets.QPushButton(self.verticalLayoutWidget)
        self.pushButton_3.setObjectName("pushButton_3")
        self.verticalLayout.addWidget(self.pushButton_3)
        spacerItem = QtWidgets.QSpacerItem(20, 40, QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Expanding)
        self.verticalLayout.addItem(spacerItem)
        self.verticalLayout.setStretch(0, 2)
        self.verticalLayout.setStretch(1, 2)
        self.verticalLayout.setStretch(2, 2)
        self.verticalLayout.setStretch(3, 2)
        self.verticalLayout.setStretch(4, 2)
        self.verticalLayout.setStretch(5, 4)
        self.verticalLayoutWidget_2 = QtWidgets.QWidget(self.centralwidget)
        self.verticalLayoutWidget_2.setGeometry(QtCore.QRect(240, 0, 881, 781))
        self.verticalLayoutWidget_2.setObjectName("verticalLayoutWidget_2")
        self.verticalLayout_2 = QtWidgets.QVBoxLayout(self.verticalLayoutWidget_2)
        self.verticalLayout_2.setContentsMargins(0, 0, 0, 0)
        self.verticalLayout_2.setObjectName("verticalLayout_2")
        self.webView = QWebEngineView(self.verticalLayoutWidget_2)
        self.webView.setObjectName("webView")
        self.webView.load(QUrl(url))
        self.verticalLayout_2.addWidget(self.webView)
        MainWindow.setCentralWidget(self.centralwidget)

        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "ML Stock"))
        self.label.setText(_translate("MainWindow", "ML STOCK RADAR"))
        self.pushButton_4.setText(_translate("MainWindow", "Dashboard"))
        self.pushButton.setText(_translate("MainWindow", "Graph"))
        self.pushButton_2.setText(_translate("MainWindow", "Financial"))
        self.pushButton_3.setText(_translate("MainWindow", "News"))


if __name__ == "__main__":
    import sys
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Ui_MainWindow()
    ui.setupUi(MainWindow)
    MainWindow.show()
    sys.exit(app.exec_())




[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


SystemExit: 0

c:\Users\1\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.



In [1]:
from PyQt5 import QtCore, QtWidgets, QtWebEngineWidgets
import plotly.express as px


class Widget(QtWidgets.QWidget):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.button = QtWidgets.QPushButton('Plot', self)
        self.browser = QtWebEngineWidgets.QWebEngineView(self)

        vlayout = QtWidgets.QVBoxLayout(self)
        vlayout.addWidget(self.button, alignment=QtCore.Qt.AlignHCenter)
        vlayout.addWidget(self.browser)

        self.button.clicked.connect(self.show_graph)
        self.resize(1000,800)

    def show_graph(self):
        df = px.data.tips()
        fig = px.box(df, x="day", y="total_bill", color="smoker")
        fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
        self.browser.setHtml(fig.to_html(include_plotlyjs='cdn'))

if __name__ == "__main__":
    app = QtWidgets.QApplication([])
    widget = Widget()
    widget.show()
    app.exec()
    

In [53]:
from jupyter_dash import JupyterDash
from dash import Dash, html, dcc, Input, Output, callback , State, ctx
import plotly.express as px
import pandas as pd

import numpy as np
import pandas as pd
import plotly
from plotly.subplots import make_subplots

#Data Source
import yfinance as yf

#Data viz
import plotly.graph_objs as go
import sqlite3
import datetime


def getLastDate(period,ticker):
    conn = sqlite3.connect("stock.sqlite")
    cur = conn.cursor()
    # Query last element of stock in database
    if period == 'Hour':
        query = "SELECT * FROM stock_table_hr WHERE `ticker` = '%s'" % ticker
    elif period == 'Day':
        query = "SELECT * FROM stock_table_d WHERE `ticker` = '%s'" % ticker
    elif period == 'Mount':
        query = "SELECT * FROM stock_table_mo WHERE `ticker` = '%s'" % ticker
    else:
        return False
    r_df = pd.read_sql(query, conn)
    # Cut data to get only datatime
    last = r_df.tail(1).Datetime.to_string().split()
    LastDate = last[1].split()[0].split('-')
    cur.close()
    return LastDate,r_df

def getDiffDay(period,ticker):
    LastDate,r_df = getLastDate(period,ticker)
    if LastDate == False:
        return False
    # Get datetime for now
    x = datetime.datetime.now()
    count = 0
    DayM = 0
    DayMo365 = {'1':31,'2':28,'3':31,'4':30,'5':31,'6':30,'7':31,'8':31,'9':30,'10':31,'11':30,'12':31}
    DiffMo = int(x.month) - int(LastDate[1])
    DiffYe = int(x.year) - int(LastDate[0])
    # Get differend day for dowload stock
    if DiffYe == 0:
        if DiffMo == 0:
            DiffDay = int(x.day) - int(LastDate[2])
            if DiffDay != 0:pass
        elif DiffMo != 0 :
            for u in range(DiffMo):
                DayM = DayM + DayMo365[str(int(LastDate[1])+count)]
                count += 1
            DiffDay = DayM - int(LastDate[2]) + int(x.day)
    elif DiffYe != 0:
        dayly = 0
        dayn = 0
        for j in range(1,int(LastDate[1])):
            dayly = dayly + DayMo365[str(j)]
        for i in range(1,int(x.month)):
            dayn = dayn + DayMo365[str(i)]
        DiffDay = (365*DiffYe) - dayly + dayn - int(LastDate[2]) + int(x.day)   
    DiffDay = str(DiffDay) + 'd'
    return DiffDay
    
    
def update(period,ticker):
    LastDate,r_df = getLastDate(period,ticker)
    DiffDay = getDiffDay(period,ticker)
    if DiffDay == False:
        return False
    down = 0
    count = 0
    conn = sqlite3.connect("stock.sqlite")
    # Select period to download
    if period == 'Hour':data = yf.download(tickers=ticker, period=DiffDay, interval='1h')
    elif period == 'Day':data = yf.download(tickers=ticker, period=DiffDay, interval='1d')
    elif period == 'Mount':data = yf.download(tickers=ticker, period=DiffDay, interval='1mo')
    # Get number of extra stock
    for i in data.index.day:
        if data.index.year[count] == int(LastDate[0]):
            if data.index.month[count] == int(LastDate[1]):
                if i == int(LastDate[2])+1 or i == int(LastDate[2])+2 or i == int(LastDate[2])+3:
                    break
        count += 1
    ok = r_df.tail(1).Datetime.to_string().split()[2]
    if ok == '10:00:00':down = 5
    elif ok == '11:00:00':down = 4
    elif ok == '12:00:00':down = 3
    elif ok == '14:00:00':down = 2
    elif ok == '15:00:00':down = 1
    elif ok == '16:00:00':down = 0
    # Cut extra stock off
    count = count - down
    data['ticker'] = ticker
    data = data.iloc[count:,:]
    data.index.rename('Datetime', inplace=True)
    # Save to sqlite
    # Select period to download
    if period == 'Hour':data.to_sql('stock_table_hr',con=conn,if_exists='append',index=True)
    elif period == 'Day':data.to_sql('stock_table_d',con=conn,if_exists='append',index=True)
    elif period == 'Mount':data.to_sql('stock_table_mo',con=conn,if_exists='append',index=True)
    return data

# a = update('Hour','PTT.BK')
# print(a)
# print(a.index.month)
# getDiffDay('Hour','AOT.BK')
# b = getLastDate('Hour','AOT.BK')
# int(b[0][0])

[*********************100%***********************]  1 of 1 completed
                      Open   High    Low  Close  Adj Close    Volume  ticker
Datetime                                                                    
2023-01-09 10:00:00  32.75  33.00  32.75  33.00      33.00         0  PTT.BK
2023-01-09 11:00:00  33.00  33.00  32.75  32.75      32.75   2031247  PTT.BK
2023-01-09 12:00:00  33.00  33.00  32.75  33.00      33.00   2711964  PTT.BK
2023-01-09 14:00:00  33.00  33.00  32.75  32.75      32.75   2368997  PTT.BK
2023-01-09 15:00:00  33.00  33.25  32.75  33.00      33.00  17004500  PTT.BK
...                    ...    ...    ...    ...        ...       ...     ...
2023-01-24 11:00:00  33.75  33.75  33.50  33.75      33.75    630045  PTT.BK
2023-01-24 12:00:00  33.75  33.75  33.50  33.50      33.50    285526  PTT.BK
2023-01-24 14:00:00  33.50  33.75  33.50  33.50      33.50    880784  PTT.BK
2023-01-24 15:00:00  33.75  33.75  33.50  33.75      33.75   1658209  PTT.BK
2023-01

In [2]:
data = yf.download(tickers='PTT.BK', period='10d', interval='1h')
data.index
# data.index = data.index.split()
# data

[*********************100%***********************]  1 of 1 completed


DatetimeIndex(['2023-01-12 10:00:00', '2023-01-12 11:00:00',
               '2023-01-12 12:00:00', '2023-01-12 14:00:00',
               '2023-01-12 15:00:00', '2023-01-12 16:00:00',
               '2023-01-13 10:00:00', '2023-01-13 11:00:00',
               '2023-01-13 12:00:00', '2023-01-13 14:00:00',
               '2023-01-13 15:00:00', '2023-01-13 16:00:00',
               '2023-01-16 10:00:00', '2023-01-16 11:00:00',
               '2023-01-16 12:00:00', '2023-01-16 14:00:00',
               '2023-01-16 15:00:00', '2023-01-16 16:00:00',
               '2023-01-17 10:00:00', '2023-01-17 11:00:00',
               '2023-01-17 12:00:00', '2023-01-17 14:00:00',
               '2023-01-17 15:00:00', '2023-01-17 16:00:00',
               '2023-01-18 10:00:00', '2023-01-18 11:00:00',
               '2023-01-18 12:00:00', '2023-01-18 14:00:00',
               '2023-01-18 15:00:00', '2023-01-18 16:00:00',
               '2023-01-19 10:00:00', '2023-01-19 11:00:00',
               '2023-01-